In [251]:
import pandas as pd
import pysam
import sys
import copy
import glob
import matplotlib.pyplot as plt
from Bio.Seq import Seq

In [314]:
cr_out = '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs'
#cr_out = '/SGRNJ06/bioinfo/PROJ04/PROJ_22.10X/P22062202_VDJ/20220712/WCM-20220617-B_BCR/WCM-20220617-B_VDJ_10X/WCM-20220617-B_VDJ_10X/outs'

In [271]:
    airr = pd.read_csv(f'{cr_out}/airr_rearrangement.tsv',sep='\t')
    contig = pd.read_csv(f'{cr_out}/filtered_contig_annotations.csv')
    clonotype = pd.read_csv(f'{cr_out}/clonotypes.csv')
    

In [268]:
    del clonotype['proportion']
    clonotype['cdr3s_aa'] = clonotype['cdr3s_aa'].apply(lambda x:x.split(';'))
    clonotype['cdr3s_nt'] = clonotype['cdr3s_nt'].apply(lambda x:x.split(';'))
    

In [275]:
clonotype['idnum'] = clonotype['clonotype_id'].apply(lambda x: int(x[9:]))

In [279]:
clonotype.sort_values(by='idnum', inplace=True)
del clonotype['idnum']

In [280]:
clonotype

,clonotype_id,frequency,proportion,cdr3s_aa,cdr3s_nt
0,clonotype1,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC
1,clonotype2,82,0.006389,IGL:CALWYSTHYVF,IGL:TGTGCTCTATGGTACAGCACCCATTATGTTTTC
2,clonotype3,71,0.005532,IGL:CALWYSNHWVF,IGL:TGTGCTCTATGGTACAGCAACCATTGGGTGTTC
4,clonotype4,62,0.004831,IGK:CQHHYGTPYTF,IGK:TGTCAACATCATTATGGTACTCCGTACACGTTC
3,clonotype5,62,0.004831,IGK:CQQYNSYPLTF,IGK:TGTCAGCAATATAACAGCTATCCTCTCACGTTC
...,...,...,...,...,...
4995,clonotype9618,1,0.000078,IGK:CQQWSSNPPTF;IGL:CALWYSNHWVV,IGK:TGCCAGCAGTGGAGTAGTAACCCACCCACGTTC;IGL:TGTG...
4996,clonotype9619,1,0.000078,IGH:CAKGYYGSSYNWFAYW;IGK:CQHFWSTPLTF,IGH:TGTGCAAAGGGCTACTACGGTAGTAGCTACAATTGGTTTGCT...
626,clonotype9620,1,0.000078,IGH:CARWGMITCYAMDYW;IGL:CALWYSTHFVF,IGH:TGTGCAAGATGGGGGATGATTACTTGCTATGCTATGGACTAC...
625,clonotype9621,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...


In [256]:
    aa_list = clonotype['cdr3s_aa'].tolist()
    aa_dict = {}
    for i,j in enumerate(aa_list,1):
        aa_dict[i] = j
    

In [ ]:
aa_dict

In [243]:
    IGH_cdr3 = dict()
    IGL_cdr3 = dict()
    for key,value in aa_dict.items():
        if len(value)==2: 
            for i in value:
                if i.startswith('IGH'):
                    IGH_cdr3['clonotype'+str(key)]= (i.split(':')[-1])
                else:
                    IGL_cdr3['clonotype'+str(key)]= (i.split(':')[-1])
    df_IGH_cdr3 = pd.DataFrame(list(IGH_cdr3.items()),columns=['clonotype_id', 'IGH_cdr3'])
    df_IGL_cdr3 = pd.DataFrame(list(IGL_cdr3.items()),columns=['clonotype_id', 'IGLorK_cdr3'])

    df_merge = pd.merge(clonotype,df_IGH_cdr3,on='clonotype_id',how='inner')
    df_merge = pd.merge(df_merge, df_IGL_cdr3,on='clonotype_id',how='inner')
    del df_merge['cdr3s_aa']
    
    nt_list = clonotype['cdr3s_nt'].tolist()
    nt_dict = {}
    for i,j in enumerate(nt_list,1):
        nt_dict[i] = j
    IGH_cdr3_nt = dict()
    IGL_cdr3_nt = dict()
    for key,value in nt_dict.items():
        for i in value:
            if i.startswith('IGH'):
                IGH_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])
            else:
                IGL_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])

    df_IGH_cdr3_nt = pd.DataFrame(list(IGH_cdr3_nt.items()),columns=['clonotype_id', 'IGH_cdr3_nt'])
    df_IGL_cdr3_nt = pd.DataFrame(list(IGL_cdr3_nt.items()),columns=['clonotype_id', 'IGLorK_cdr3_nt'])
    df_merge = pd.merge(df_merge,df_IGH_cdr3_nt,on='clonotype_id',how='inner')
    df_merge = pd.merge(df_merge,df_IGL_cdr3_nt,on='clonotype_id',how='inner')
    del df_merge['cdr3s_nt']
    
    df_merge['Clonotype'] = df_merge['IGH_cdr3'] + '_' + df_merge['IGLorK_cdr3']
    df_merge =df_merge[['clonotype_id','Clonotype','frequency','IGH_cdr3','IGH_cdr3_nt','IGLorK_cdr3','IGLorK_cdr3_nt']]

    airr = airr[['clone_id','sequence_id','sequence','sequence_aa']]
    airr.rename(columns={'sequence_id':'contig_id'},inplace=True)

In [235]:
airr

,clone_id,contig_id,sequence,sequence_aa
0,NaN,AAACCTGAGAAACCGC-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...
1,NaN,AAACCTGAGAAACGAG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...
2,NaN,AAACCTGAGAAACGCC-1_contig_1,GTAAGAGCAAGTATGTACCGTTCCAAGCAGTGGTATCAACGCAGAG...,METDTLLLWVLLLWVPGSTGDIVLTQSPASLAVSLGQRATISCRAS...
3,NaN,AAACCTGAGAAACGCC-1_contig_2,AGTCCTGGATTTGAGTTCCTCACATTCAGTGATGAGCACTGAACAC...,MNFGLSLIFLVLVLKGVLCEVKLVESGGGLVQPGGSLKLSCAASGF...
4,NaN,AAACCTGAGAAAGTGG-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCTCATCTC...,MGWSYIILFLVATATDVHSQVQLQQPGAELVKPGASVKLSCKASGY...
...,...,...,...,...
23256,NaN,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...
23257,NaN,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...
23258,NaN,AGTAGTCTCACCACCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...
23259,NaN,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...


In [236]:
contig = contig[contig['productive']==True]

In [237]:
    airr

,clone_id,contig_id,sequence,sequence_aa
0,NaN,AAACCTGAGAAACCGC-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...
1,NaN,AAACCTGAGAAACGAG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...
2,NaN,AAACCTGAGAAACGCC-1_contig_1,GTAAGAGCAAGTATGTACCGTTCCAAGCAGTGGTATCAACGCAGAG...,METDTLLLWVLLLWVPGSTGDIVLTQSPASLAVSLGQRATISCRAS...
3,NaN,AAACCTGAGAAACGCC-1_contig_2,AGTCCTGGATTTGAGTTCCTCACATTCAGTGATGAGCACTGAACAC...,MNFGLSLIFLVLVLKGVLCEVKLVESGGGLVQPGGSLKLSCAASGF...
4,NaN,AAACCTGAGAAAGTGG-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCTCATCTC...,MGWSYIILFLVATATDVHSQVQLQQPGAELVKPGASVKLSCKASGY...
...,...,...,...,...
23256,NaN,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...
23257,NaN,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...
23258,NaN,AGTAGTCTCACCACCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...
23259,NaN,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...


In [238]:
    if str(airr.clone_id[0]) =='nan':
        airr = pd.merge(airr,contig[['contig_id','raw_clonotype_id']],on='contig_id',how='inner')
        airr.clone_id = airr.raw_clonotype_id
        del airr['raw_clonotype_id']

In [239]:
airr

,clone_id,contig_id,sequence,sequence_aa
0,clonotype253,AAACCTGAGAAACCGC-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...
1,clonotype46,AAACCTGAGAAACGAG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...
2,clonotype493,AAACCTGAGAAACGCC-1_contig_1,GTAAGAGCAAGTATGTACCGTTCCAAGCAGTGGTATCAACGCAGAG...,METDTLLLWVLLLWVPGSTGDIVLTQSPASLAVSLGQRATISCRAS...
3,clonotype493,AAACCTGAGAAACGCC-1_contig_2,AGTCCTGGATTTGAGTTCCTCACATTCAGTGATGAGCACTGAACAC...,MNFGLSLIFLVLVLKGVLCEVKLVESGGGLVQPGGSLKLSCAASGF...
4,clonotype494,AAACCTGAGAAAGTGG-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCTCATCTC...,MGWSYIILFLVATATDVHSQVQLQQPGAELVKPGASVKLSCKASGY...
...,...,...,...,...
23256,clonotype9621,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...
23257,clonotype9621,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...
23258,clonotype51,AGTAGTCTCACCACCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...
23259,clonotype9622,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...


In [240]:
    contig = contig[contig['productive']==True]
    contig.sort_values('reads',ascending=False,inplace=True)
    contig= contig[['contig_id','chain']]
    df_merge1 = pd.merge(airr,contig,on='contig_id')
    df_merge1.rename(columns={'clone_id':'clonotype_id'},inplace=True)
    df_merge1['idnum'] = df_merge1['clonotype_id'].apply(lambda x: int(x[9:]))
    df_merge1.sort_values(['idnum','chain'],inplace=True)
    del df_merge1['idnum']

In [241]:
df_merge

,clonotype_id,Clonotype,frequency,IGH_cdr3,IGH_cdr3_nt,IGLorK_cdr3,IGLorK_cdr3_nt
0,clonotype91,CARYYYGYGDAMDYW_CQQHYSTPYTF,11,CARYYYGYGDAMDYW,TGTGCAAGATATTACTACGGCTACGGGGATGCTATGGACTACTGG,CQQHYSTPYTF,TGTCAGCAACATTATAGCACTCCGTACACGTTC
1,clonotype185,CAREGFAYW_CALWYSNHLVF,5,CAREGFAYW,TGTGCAAGAGAGGGGTTTGCTTACTGG,CALWYSNHLVF,TGTGCTCTATGGTACAGCAACCATTTGGTGTTC
2,clonotype251,CARYYGNYYAMDYW_CHQRSSYPWTF,4,CARYYGNYYAMDYW,TGTGCAAGATACTATGGTAACTACTATGCTATGGACTACTGG,CHQRSSYPWTF,TGCCATCAGCGGAGTAGTTACCCATGGACGTTC
3,clonotype249,CARTGTGMDYW_CALWYSTHYVF,4,CARTGTGMDYW,TGTGCAAGAACTGGGACCGGTATGGACTACTGG,CALWYSTHYVF,TGTGCTCTATGGTACAGCACCCATTATGTTTTC
4,clonotype229,CMRYGSSYWYFDVW_CLQHGESPFTF,4,CMRYGSSYWYFDVW,TGTATGAGATACGGTAGTAGCTACTGGTACTTCGATGTCTGG,CLQHGESPFTF,TGTCTACAGCATGGTGAGAGCCCATTCACGTTC
...,...,...,...,...,...,...,...
6267,clonotype4744,CARHPYDYDGRSYAMDYW_CQQYSSYLTF,1,CARHPYDYDGRSYAMDYW,TGTGCAAGACATCCCTATGATTACGACGGAAGGTCCTATGCTATGG...,CQQYSSYLTF,TGTCAGCAATATAGCAGCTATCTCACGTTC
6268,clonotype4743,CARQKTARATSWFAYW_CAQNLELPLTF,1,CARQKTARATSWFAYW,TGTGCCAGACAAAAGACAGCTCGGGCTACGTCCTGGTTTGCTTACTGG,CAQNLELPLTF,TGTGCTCAAAATCTAGAACTTCCGCTCACGTTC
6269,clonotype4742,CAWSHYGYDEVYW_CWQGTHFPWTF,1,CAWSHYGYDEVYW,TGTGCATGGTCCCACTATGGTTACGACGAGGTGTACTGG,CWQGTHFPWTF,TGCTGGCAAGGTACACATTTTCCGTGGACGTTC
6270,clonotype4740,CARGRYDYAMDYW_CQHHYGTPYTF,1,CARGRYDYAMDYW,TGTGCTAGAGGGAGGTACGACTATGCTATGGACTACTGG,CQHHYGTPYTF,TGTCAACATCATTATGGTACTCCGTACACGTTC


In [ ]:
    target_contigid = df_merge['clonotype_id'].tolist()
    df_merge1 = df_merge1[df_merge1['clonotype_id'].isin(target_contigid)]
    df_merge1.drop_duplicates(['clonotype_id','chain'],inplace=True)
    df_merge1_IGH = df_merge1[df_merge1['chain']=='IGH']
    df_merge1_IGL = df_merge1[df_merge1['chain']!='IGH']
    del df_merge1_IGH['chain']
    del df_merge1_IGL['chain']
    df_merge1_IGH.rename(columns={'contig_id':'contig_id_IGH'},inplace=True)
    df_merge1_IGL.rename(columns={'contig_id':'contig_id_IGLorKs'},inplace=True)
    df_merge1_IGH.rename(columns={'sequence':'IGH_nt'},inplace=True)
    df_merge1_IGL.rename(columns={'sequence':'IGLorK_nt'},inplace=True)
    df_merge1_IGH.rename(columns={'sequence_aa':'IGH_aa'},inplace=True)
    df_merge1_IGL.rename(columns={'sequence_aa':'IGLorK_aa'},inplace=True)

In [ ]:
df_merge_f

In [ ]:
    df_merge_f = pd.merge(df_merge,df_merge1_IGH,on='clonotype_id')

In [ ]:
    df_merge_f = pd.merge(df_merge_f,df_merge1_IGL,on='clonotype_id')
    df_merge_f.rename(columns={'frequency':'Frequency'},inplace=True)
    df_merge_f = df_merge_f[['clonotype_id','Clonotype','Frequency','IGH_cdr3','IGH_cdr3_nt','contig_id_IGH','IGH_aa','IGH_nt',
                            'IGLorK_cdr3','IGLorK_cdr3_nt','contig_id_IGLorKs','IGLorK_aa','IGLorK_nt']]

In [ ]:
df_merge_f

In [ ]:
df_merge_f[df_merge_f['clonotype_id']=='ClonotypeID90']

In [282]:
','.join(str([0.1]*2))

'[,0,.,1,,, ,0,.,1,]'

[0.1, 0.1, 0.1, 0.1, 0.1]

In [325]:
    airr = pd.read_csv(f'{cr_out}/airr_rearrangement.tsv',sep='\t')
    contig = pd.read_csv(f'{cr_out}/filtered_contig_annotations.csv')
    clonotype = pd.read_csv(f'{cr_out}/clonotypes.csv')
    
    contig = contig[contig['productive']==True]
    contig_for_plot = copy.deepcopy(contig)

In [326]:
contig

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,AAACCTGAGAAACCGC-1,True,AAACCTGAGAAACCGC-1_contig_1,True,483,IGK,IGKV4-70,None,IGKJ2,IGKC,True,True,CHQRSSYTF,TGCCATCAGCGGAGTAGTTACACGTTC,777,33,clonotype253,clonotype253_consensus_1
5,AAACCTGAGAAACGAG-1,True,AAACCTGAGAAACGAG-1_contig_1,True,478,IGL,IGLV1,None,IGLJ1,IGLC1,True,True,CALWYSNHWVF,TGTGCTCTATGGTACAGCAACCACTGGGTGTTC,794,21,clonotype46,clonotype46_consensus_1
7,AAACCTGAGAAACGCC-1,True,AAACCTGAGAAACGCC-1_contig_1,True,523,IGK,IGKV3-5,None,IGKJ5,IGKC,True,True,CQQSNEDPPTF,TGTCAGCAAAGTAATGAGGATCCTCCCACGTTC,378,22,clonotype493,clonotype493_consensus_2
8,AAACCTGAGAAACGCC-1,True,AAACCTGAGAAACGCC-1_contig_2,True,528,IGH,IGHV5-12,None,IGHJ3,IGHM,True,True,CARQRDYGSSSWFAYW,TGTGCAAGACAGAGGGACTACGGTAGTAGCTCCTGGTTTGCTTACTGG,21,5,clonotype493,clonotype493_consensus_1
9,AAACCTGAGAAAGTGG-1,True,AAACCTGAGAAAGTGG-1_contig_1,True,725,IGH,IGHV1-53,None,IGHJ4,IGHD,True,True,CASWGGKYAMDYW,TGTGCAAGTTGGGGAGGTAAATATGCTATGGACTACTGG,186,8,clonotype494,clonotype494_consensus_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45341,AGGTCCGCACCTTGTC-1,True,AGGTCCGCACCTTGTC-1_contig_1,True,539,IGH,IGHV1-55,None,IGHJ3,IGHM,True,True,CTRGGLRRGFAYW,TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG,24,1,clonotype9621,clonotype9621_consensus_2
45342,AGGTCCGCACCTTGTC-1,True,AGGTCCGCACCTTGTC-1_contig_2,True,474,IGK,IGKV6-15,None,IGKJ2,IGKC,True,True,CQQYNSYPYTF,TGTCAGCAATATAACAGCTATCCGTACACGTTC,41,7,clonotype9621,clonotype9621_consensus_1
45343,AGTAGTCTCACCACCT-1,True,AGTAGTCTCACCACCT-1_contig_1,True,483,IGK,IGKV15-103,None,IGKJ1,IGKC,True,True,CQQGQSYPWTF,TGTCAACAGGGTCAAAGTTATCCGTGGACGTTC,34,3,clonotype51,clonotype51_consensus_1
45344,ATCACGACAAGCCGCT-1,True,ATCACGACAAGCCGCT-1_contig_1,True,483,IGK,IGKV15-103,None,IGKJ2,IGKC,True,True,CQQGQSYPYTF,TGTCAACAGGGTCAAAGTTATCCGTACACGTTC,54,4,clonotype9622,clonotype9622_consensus_2


In [306]:
    contig = contig[['contig_id','raw_clonotype_id','cdr3','reads']]
    contig.rename(columns={'contig_id':'sequence_id'},inplace=True)
    df_merge = pd.merge(airr,contig,on='sequence_id',how='inner')
    del df_merge['clone_id']
    df_merge.rename(columns={'raw_clonotype_id':'clonotype_id'},inplace=True)
    
    df_merge_f = pd.merge(df_merge,clonotype,on='clonotype_id')
    df_merge_f['chain'] = df_merge_f['v_call'].apply(lambda x:x[:3])
    df_merge_f['id_num'] = df_merge_f['clonotype_id'].apply(lambda x: int(x[9:]))

In [311]:
df_merge_f

,cell_id,sequence_id,sequence,sequence_aa,productive,rev_comp,v_call,v_cigar,d_call,d_cigar,...,is_cell,clonotype_id,cdr3,reads,frequency,proportion,cdr3s_aa,cdr3s_nt,chain,id_num
0,AAACCTGAGAAACCGC-1,AAACCTGAGAAACCGC-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...,T,F,IGKV4-70,31S348M104S,NaN,NaN,...,T,clonotype253,CHQRSSYTF,777,3,0.000234,IGK:CHQRSSYTF,IGK:TGCCATCAGCGGAGTAGTTACACGTTC,IGK,253
1,AACACGTCAGGGTATG-1,AACACGTCAGGGTATG-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...,T,F,IGKV4-70,31S348M104S,NaN,NaN,...,T,clonotype253,CHQRSSYTF,694,3,0.000234,IGK:CHQRSSYTF,IGK:TGCCATCAGCGGAGTAGTTACACGTTC,IGK,253
2,AATCGGTAGGACCACA-1,AATCGGTAGGACCACA-1_contig_1,TTCTCTTCGCACCCGTAGCCGCATGCTGATGCTGATATCTCAAGCA...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...,T,F,IGKV4-70,193S348M104S,NaN,NaN,...,T,clonotype253,CHQRSSYTF,196,3,0.000234,IGK:CHQRSSYTF,IGK:TGCCATCAGCGGAGTAGTTACACGTTC,IGK,253
3,AAACCTGAGAAACGAG-1,AAACCTGAGAAACGAG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,T,F,IGLV1,40S351M87S,NaN,NaN,...,T,clonotype46,CALWYSNHWVF,794,18,0.001402,IGL:CALWYSNHWVF,IGL:TGTGCTCTATGGTACAGCAACCACTGGGTGTTC,IGL,46
4,AAAGATGGTGTCGCTG-1,AAAGATGGTGTCGCTG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,T,F,IGLV1,40S351M83S,NaN,NaN,...,T,clonotype46,CALWYSNHWVF,3157,18,0.001402,IGL:CALWYSNHWVF,IGL:TGTGCTCTATGGTACAGCAACCACTGGGTGTTC,IGL,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23256,AGGGATGTCCTATTCA-1,AGGGATGTCCTATTCA-1_contig_2,TAGTGACAATCGAGGTGCTAGATCGGAAGAGCGTCGTGTAGGAGCA...,MAWVWTLLFLMAAAQSAQAQIQLVQSGPELKKPGETVKISCKASGY...,T,F,IGHV9-3,136S351M90S,NaN,NaN,...,T,clonotype9620,CARWGMITCYAMDYW,21,1,0.000078,IGH:CARWGMITCYAMDYW;IGL:CALWYSTHFVF,IGH:TGTGCAAGATGGGGGATGATTACTTGCTATGCTATGGACTAC...,IGH,9620
23257,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...,T,F,IGHV1-55,104S351M84S,NaN,NaN,...,T,clonotype9621,CTRGGLRRGFAYW,24,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGH,9621
23258,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,T,F,IGKV6-15,19S347M108S,NaN,NaN,...,T,clonotype9621,CQQYNSYPYTF,41,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGK,9621
23259,ATCACGACAAGCCGCT-1,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,T,F,IGKV15-103,28S347M108S,NaN,NaN,...,T,clonotype9622,CQQGQSYPYTF,54,1,0.000078,IGK:CQQGQSYPYTF;IGL:CALWYSNHWVF,IGK:TGTCAACAGGGTCAAAGTTATCCGTACACGTTC;IGL:TGTG...,IGK,9622


In [313]:
    df_merge_f.sort_values(['id_num','chain','reads'],ascending=[True,True,False])
    #df_merge_f.sort_values(['id_num','chain','reads'])

,cell_id,sequence_id,sequence,sequence_aa,productive,rev_comp,v_call,v_cigar,d_call,d_cigar,...,is_cell,clonotype_id,cdr3,reads,frequency,proportion,cdr3s_aa,cdr3s_nt,chain,id_num
904,AAACCTGCATTTCAGG-1,AAACCTGCATTTCAGG-1_contig_1,AGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCA...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,208S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,5570,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
943,AACCATGGTTATTCTC-1,AACCATGGTTATTCTC-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGGCAGTGCAG...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,206S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,1948,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
906,AAACGGGAGTTTCCTT-1,AAACGGGAGTTTCCTT-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCAAGCCGTA...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,218S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,1550,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
914,AAAGATGAGTGAACAT-1,AAAGATGAGTGAACAT-1_contig_1,AACGCAGAGTACATGGGGAGATTGTGAGCCATTGTAATTGAAGTCA...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,61S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,1465,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
959,AACTCCCGTCTAACGT-1,AACTCCCGTCTAACGT-1_contig_1,CCCCCCCGCCCGCCCCCCTGGTTTATGATCCGGCGACCACCGTGAT...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,255S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,1355,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23255,AGGGATGTCCTATTCA-1,AGGGATGTCCTATTCA-1_contig_1,TAATAGACTTGGTTTGTGAATTATGGCCTGGACTTCACTTATACTC...,MAWTSLILSLLALCSGASSQAVVTQESALTTSPGGTVILTCRSSTG...,T,F,IGLV2,22S351M87S,NaN,NaN,...,T,clonotype9620,CALWYSTHFVF,17,1,0.000078,IGH:CARWGMITCYAMDYW;IGL:CALWYSTHFVF,IGH:TGTGCAAGATGGGGGATGATTACTTGCTATGCTATGGACTAC...,IGL,9620
23257,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...,T,F,IGHV1-55,104S351M84S,NaN,NaN,...,T,clonotype9621,CTRGGLRRGFAYW,24,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGH,9621
23258,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,T,F,IGKV6-15,19S347M108S,NaN,NaN,...,T,clonotype9621,CQQYNSYPYTF,41,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGK,9621
23259,ATCACGACAAGCCGCT-1,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,T,F,IGKV15-103,28S347M108S,NaN,NaN,...,T,clonotype9622,CQQGQSYPYTF,54,1,0.000078,IGK:CQQGQSYPYTF;IGL:CALWYSNHWVF,IGK:TGTCAACAGGGTCAAAGTTATCCGTACACGTTC;IGL:TGTG...,IGK,9622


In [296]:
    df_merge_f.sort_values(['id_num','chain'],inplace=True)

/Personal/chenjunjie/.local/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [298]:
df_merge_f.columns

Index(['cell_id', 'sequence_id', 'sequence', 'sequence_aa', 'productive',
       'rev_comp', 'v_call', 'v_cigar', 'd_call', 'd_cigar', 'j_call',
       'j_cigar', 'c_call', 'c_cigar', 'sequence_alignment',
       'germline_alignment', 'junction', 'junction_aa', 'junction_length',
       'junction_aa_length', 'v_sequence_start', 'v_sequence_end',
       'd_sequence_start', 'd_sequence_end', 'j_sequence_start',
       'j_sequence_end', 'c_sequence_start', 'c_sequence_end',
       'consensus_count', 'duplicate_count', 'is_cell', 'clonotype_id', 'cdr3',
       'reads', 'frequency', 'proportion', 'cdr3s_aa', 'cdr3s_nt', 'chain',
       'id_num'],
      dtype='object')

In [299]:
df_merge_f

,cell_id,sequence_id,sequence,sequence_aa,productive,rev_comp,v_call,v_cigar,d_call,d_cigar,...,is_cell,clonotype_id,cdr3,reads,frequency,proportion,cdr3s_aa,cdr3s_nt,chain,id_num
901,AAACCTGAGTATCTCG-1,AAACCTGAGTATCTCG-1_contig_1,AATACCACCAATGCTAATACGTAGTCATCACGTTGAGACCTACATG...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,162S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,654,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
902,AAACCTGCAAATACAG-1,AAACCTGCAAATACAG-1_contig_1,GGTGAGTTCGTAGCCGCATGCTGATATGCCAGTCTCAAGCACGTGG...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,160S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,769,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
903,AAACCTGCATCGACGC-1,AAACCTGCATCGACGC-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCGGTTGTC...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,206S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,591,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
904,AAACCTGCATTTCAGG-1,AAACCTGCATTTCAGG-1_contig_1,AGATCTACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCA...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,208S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,5570,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
905,AAACGGGAGCCACTAT-1,AAACGGGAGCCACTAT-1_contig_1,TACAGGATTCAGCTTCTGACTACGTACTAGTCATATGGCTGTCACC...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,T,F,IGKV10-96,160S347M108S,NaN,NaN,...,T,clonotype1,CQQGNTLPWTF,1079,89,0.006934,IGK:CQQGNTLPWTF,IGK:TGCCAACAGGGTAATACGCTTCCGTGGACGTTC,IGK,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23255,AGGGATGTCCTATTCA-1,AGGGATGTCCTATTCA-1_contig_1,TAATAGACTTGGTTTGTGAATTATGGCCTGGACTTCACTTATACTC...,MAWTSLILSLLALCSGASSQAVVTQESALTTSPGGTVILTCRSSTG...,T,F,IGLV2,22S351M87S,NaN,NaN,...,T,clonotype9620,CALWYSTHFVF,17,1,0.000078,IGH:CARWGMITCYAMDYW;IGL:CALWYSTHFVF,IGH:TGTGCAAGATGGGGGATGATTACTTGCTATGCTATGGACTAC...,IGL,9620
23257,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...,T,F,IGHV1-55,104S351M84S,NaN,NaN,...,T,clonotype9621,CTRGGLRRGFAYW,24,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGH,9621
23258,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,T,F,IGKV6-15,19S347M108S,NaN,NaN,...,T,clonotype9621,CQQYNSYPYTF,41,1,0.000078,IGH:CTRGGLRRGFAYW;IGK:CQQYNSYPYTF,IGH:TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG;IG...,IGK,9621
23259,ATCACGACAAGCCGCT-1,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,T,F,IGKV15-103,28S347M108S,NaN,NaN,...,T,clonotype9622,CQQGQSYPYTF,54,1,0.000078,IGK:CQQGQSYPYTF;IGL:CALWYSNHWVF,IGK:TGTCAACAGGGTCAAAGTTATCCGTACACGTTC;IGL:TGTG...,IGK,9622


In [315]:
 
    airr = pd.read_csv(f'{cr_out}/airr_rearrangement.tsv',sep='\t')
    contig = pd.read_csv(f'{cr_out}/filtered_contig_annotations.csv')
    clonotype = pd.read_csv(f'{cr_out}/clonotypes.csv')
    
    # cr 4.0
    if str(airr.clone_id[0]) =='nan':
        clonotype['idnum'] = clonotype['clonotype_id'].apply(lambda x: int(x[9:]))
        clonotype.sort_values(by='idnum', inplace=True)
        del clonotype['idnum']
    del clonotype['proportion']
    clonotype['cdr3s_aa'] = clonotype['cdr3s_aa'].apply(lambda x:x.split(';'))
    clonotype['cdr3s_nt'] = clonotype['cdr3s_nt'].apply(lambda x:x.split(';'))
    
    aa_list = clonotype['cdr3s_aa'].tolist()
    aa_dict = {}
    for i,j in enumerate(aa_list,1):
        aa_dict[i] = j
    
    IGH_cdr3 = dict()
    IGL_cdr3 = dict()
    for key,value in aa_dict.items():
        if len(value)==2: 
            for i in value:
                if i.startswith('IGH'):
                    IGH_cdr3['clonotype'+str(key)]= (i.split(':')[-1])
                else:
                    IGL_cdr3['clonotype'+str(key)]= (i.split(':')[-1])
    df_IGH_cdr3 = pd.DataFrame(list(IGH_cdr3.items()),columns=['clonotype_id', 'IGH_cdr3'])
    df_IGL_cdr3 = pd.DataFrame(list(IGL_cdr3.items()),columns=['clonotype_id', 'IGLorK_cdr3'])

    df_merge = pd.merge(clonotype,df_IGH_cdr3,on='clonotype_id',how='inner')
    df_merge = pd.merge(df_merge, df_IGL_cdr3,on='clonotype_id',how='inner')
    del df_merge['cdr3s_aa']
    
    nt_list = clonotype['cdr3s_nt'].tolist()
    nt_dict = {}
    for i,j in enumerate(nt_list,1):
        nt_dict[i] = j
    IGH_cdr3_nt = dict()
    IGL_cdr3_nt = dict()
    for key,value in nt_dict.items():
        for i in value:
            if i.startswith('IGH'):
                IGH_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])
            else:
                IGL_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])

    df_IGH_cdr3_nt = pd.DataFrame(list(IGH_cdr3_nt.items()),columns=['clonotype_id', 'IGH_cdr3_nt'])
    df_IGL_cdr3_nt = pd.DataFrame(list(IGL_cdr3_nt.items()),columns=['clonotype_id', 'IGLorK_cdr3_nt'])
    df_merge = pd.merge(df_merge,df_IGH_cdr3_nt,on='clonotype_id',how='inner')
    df_merge = pd.merge(df_merge,df_IGL_cdr3_nt,on='clonotype_id',how='inner')
    del df_merge['cdr3s_nt']
    
    df_merge['Clonotype'] = df_merge['IGH_cdr3'] + '_' + df_merge['IGLorK_cdr3']
    df_merge =df_merge[['clonotype_id','Clonotype','frequency','IGH_cdr3','IGH_cdr3_nt','IGLorK_cdr3','IGLorK_cdr3_nt']]

    airr = airr[['clone_id','sequence_id','sequence','sequence_aa']]
    airr.rename(columns={'sequence_id':'contig_id'},inplace=True)

In [316]:
    contig = contig[contig['productive']==True]
    if str(airr.clone_id[0]) =='nan':
        airr = pd.merge(airr,contig[['contig_id','raw_clonotype_id']],on='contig_id',how='inner')
        airr.clone_id = airr.raw_clonotype_id
        del airr['raw_clonotype_id']
        

In [317]:
contig

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,AAACCTGAGAAACCGC-1,True,AAACCTGAGAAACCGC-1_contig_1,True,483,IGK,IGKV4-70,None,IGKJ2,IGKC,True,True,CHQRSSYTF,TGCCATCAGCGGAGTAGTTACACGTTC,777,33,clonotype253,clonotype253_consensus_1
5,AAACCTGAGAAACGAG-1,True,AAACCTGAGAAACGAG-1_contig_1,True,478,IGL,IGLV1,None,IGLJ1,IGLC1,True,True,CALWYSNHWVF,TGTGCTCTATGGTACAGCAACCACTGGGTGTTC,794,21,clonotype46,clonotype46_consensus_1
7,AAACCTGAGAAACGCC-1,True,AAACCTGAGAAACGCC-1_contig_1,True,523,IGK,IGKV3-5,None,IGKJ5,IGKC,True,True,CQQSNEDPPTF,TGTCAGCAAAGTAATGAGGATCCTCCCACGTTC,378,22,clonotype493,clonotype493_consensus_2
8,AAACCTGAGAAACGCC-1,True,AAACCTGAGAAACGCC-1_contig_2,True,528,IGH,IGHV5-12,None,IGHJ3,IGHM,True,True,CARQRDYGSSSWFAYW,TGTGCAAGACAGAGGGACTACGGTAGTAGCTCCTGGTTTGCTTACTGG,21,5,clonotype493,clonotype493_consensus_1
9,AAACCTGAGAAAGTGG-1,True,AAACCTGAGAAAGTGG-1_contig_1,True,725,IGH,IGHV1-53,None,IGHJ4,IGHD,True,True,CASWGGKYAMDYW,TGTGCAAGTTGGGGAGGTAAATATGCTATGGACTACTGG,186,8,clonotype494,clonotype494_consensus_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45341,AGGTCCGCACCTTGTC-1,True,AGGTCCGCACCTTGTC-1_contig_1,True,539,IGH,IGHV1-55,None,IGHJ3,IGHM,True,True,CTRGGLRRGFAYW,TGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGG,24,1,clonotype9621,clonotype9621_consensus_2
45342,AGGTCCGCACCTTGTC-1,True,AGGTCCGCACCTTGTC-1_contig_2,True,474,IGK,IGKV6-15,None,IGKJ2,IGKC,True,True,CQQYNSYPYTF,TGTCAGCAATATAACAGCTATCCGTACACGTTC,41,7,clonotype9621,clonotype9621_consensus_1
45343,AGTAGTCTCACCACCT-1,True,AGTAGTCTCACCACCT-1_contig_1,True,483,IGK,IGKV15-103,None,IGKJ1,IGKC,True,True,CQQGQSYPWTF,TGTCAACAGGGTCAAAGTTATCCGTGGACGTTC,34,3,clonotype51,clonotype51_consensus_1
45344,ATCACGACAAGCCGCT-1,True,ATCACGACAAGCCGCT-1_contig_1,True,483,IGK,IGKV15-103,None,IGKJ2,IGKC,True,True,CQQGQSYPYTF,TGTCAACAGGGTCAAAGTTATCCGTACACGTTC,54,4,clonotype9622,clonotype9622_consensus_2


In [318]:
    contig.sort_values('reads',ascending=False,inplace=True)
    contig= contig[['contig_id','chain']]
    df_merge1 = pd.merge(airr,contig,on='contig_id')
    df_merge1.rename(columns={'clone_id':'clonotype_id'},inplace=True)
    df_merge1['idnum'] = df_merge1['clonotype_id'].apply(lambda x: int(x[9:]))

In [319]:
df_merge1

,clonotype_id,contig_id,sequence,sequence_aa,chain,idnum
0,clonotype253,AAACCTGAGAAACCGC-1_contig_1,AATTAGCTAGGGACCAAAATTCAAAGACAAAATGGATTTTCAGGTG...,MDFQVQIFSFLLISASVILSRGQIVLTQSPAIMSASPGEKVTMTCS...,IGK,253
1,clonotype46,AAACCTGAGAAACGAG-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,IGL,46
2,clonotype493,AAACCTGAGAAACGCC-1_contig_1,GTAAGAGCAAGTATGTACCGTTCCAAGCAGTGGTATCAACGCAGAG...,METDTLLLWVLLLWVPGSTGDIVLTQSPASLAVSLGQRATISCRAS...,IGK,493
3,clonotype493,AAACCTGAGAAACGCC-1_contig_2,AGTCCTGGATTTGAGTTCCTCACATTCAGTGATGAGCACTGAACAC...,MNFGLSLIFLVLVLKGVLCEVKLVESGGGLVQPGGSLKLSCAASGF...,IGH,493
4,clonotype494,AAACCTGAGAAAGTGG-1_contig_1,ACACTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCTCATCTC...,MGWSYIILFLVATATDVHSQVQLQQPGAELVKPGASVKLSCKASGY...,IGH,494
...,...,...,...,...,...,...
23256,clonotype9621,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...,IGH,9621
23257,clonotype9621,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,IGK,9621
23258,clonotype51,AGTAGTCTCACCACCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,IGK,51
23259,clonotype9622,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,IGK,9622


In [320]:
    df_merge1.sort_values(['idnum','chain'],inplace=True)
    del df_merge1['idnum']

In [322]:
    target_contigid = df_merge['clonotype_id'].tolist()
    df_merge1 = df_merge1[df_merge1['clonotype_id'].isin(target_contigid)]

In [324]:
df_merge

,clonotype_id,Clonotype,frequency,IGH_cdr3,IGH_cdr3_nt,IGLorK_cdr3,IGLorK_cdr3_nt
0,clonotype90,CARYYYGYGDAMDYW_CQQHYSTPYTF,11,CARYYYGYGDAMDYW,TGTGCAAGATATTACTACGGCTACGGGGATGCTATGGACTACTGG,CQQHYSTPYTF,TGTCAGCAACATTATAGCACTCCGTACACGTTC
1,clonotype193,CAREGFAYW_CALWYSNHLVF,5,CAREGFAYW,TGTGCAAGAGAGGGGTTTGCTTACTGG,CALWYSNHLVF,TGTGCTCTATGGTACAGCAACCATTTGGTGTTC
2,clonotype212,CMRYGSSYWYFDVW_CLQHGESPFTF,4,CMRYGSSYWYFDVW,TGTATGAGATACGGTAGTAGCTACTGGTACTTCGATGTCTGG,CLQHGESPFTF,TGTCTACAGCATGGTGAGAGCCCATTCACGTTC
3,clonotype225,CARYYGNYYAMDYW_CHQRSSYPWTF,4,CARYYGNYYAMDYW,TGTGCAAGATACTATGGTAACTACTATGCTATGGACTACTGG,CHQRSSYPWTF,TGCCATCAGCGGAGTAGTTACCCATGGACGTTC
4,clonotype227,CARTGTGMDYW_CALWYSTHYVF,4,CARTGTGMDYW,TGTGCAAGAACTGGGACCGGTATGGACTACTGG,CALWYSTHYVF,TGTGCTCTATGGTACAGCACCCATTATGTTTTC
...,...,...,...,...,...,...,...
6267,clonotype9614,CASPLYYRYDGFAYW_CQHHYGTPLTF,1,CASPLYYRYDGFAYW,TGTGCAAGCCCTCTCTACTATAGGTACGACGGGTTTGCTTACTGG,CQHHYGTPLTF,TGTCAACATCATTATGGTACTCCTCTCACGTTC
6268,clonotype9616,CARSDYYGGDYW_CHQYHRSPLTF,1,CARSDYYGGDYW,TGTGCAAGATCGGATTACTACGGGGGAGACTACTGG,CHQYHRSPLTF,TGCCACCAGTATCATCGTTCCCCACTCACGTTC
6269,clonotype9619,CAKGYYGSSYNWFAYW_CQHFWSTPLTF,1,CAKGYYGSSYNWFAYW,TGTGCAAAGGGCTACTACGGTAGTAGCTACAATTGGTTTGCTTACTGG,CQHFWSTPLTF,TGTCAACATTTTTGGAGTACTCCGCTCACGTTC
6270,clonotype9620,CARWGMITCYAMDYW_CALWYSTHFVF,1,CARWGMITCYAMDYW,TGTGCAAGATGGGGGATGATTACTTGCTATGCTATGGACTACTGG,CALWYSTHFVF,TGTGCTCTATGGTACAGCACCCATTTTGTTTTC


In [328]:
import pysam

In [329]:
old = '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220718anno_seq_sc_test/anno_seq_nt.fasta'
new = '/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220718anno_seq_sc_test/1/anno_seq_nt.fasta'

In [332]:
with open(old) as f1, open(new) as f2:
    oldlines = f1.readlines()
    newlines = f2.readlines()

In [334]:
for i in range(len(oldlines)):
    if oldlines[i] != newlines[i]:
        print(oldlines[i])
        print(newlines[i])
        print('\n')

ATGGCCTGGACTTCACTTATACTCTCTCTCCTGGCTCTCTGCTCAGGAGCCAGTTCCCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTGGAACAGTCATACTCACTTGTCGCTCAAGTACTGGGGCTGTTACAACTAGTAACTATGCCAACTGGGTCCAAGAAAAACCAGATCATTTATTCACTGGTCTAATAGGTGGTACCAGCAACCGAGCTCCAGGTGTTCCTGTCAGATTCTCAGGCTCCCTGATTGGAGACAAGGCTGCCCTCACCATCACAGGGGCACAGACTGAGGATGATGCAATGTATTTCTGTGCTCTATGGTACAGCACCCATTATGTTTTCGGCGGTGGAACCAAGGTCACTGTCCTAGGTCAGCCCAAGTCCACTCCCACTCTCACCGTGTTTCCACCTTCCTCTGAGGAG

ATGGCCTGGACTTCACTTATACTCTCTCTCCTGGCTCTCTGCTCAGGAGCCAGTTCCCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTGGAACAGTCATACTCACTTGTCGCTCAAGTACTGGGGCTGTTACAACTAGTAACTATGCCAACTGGGTCCAAGAAAAACCAGATCATTTATTCACTGGTCTAATAGGTGGTACCAGCAACCGAGCTCCAGGTGTTCCTGTCAGATTCTCAGGCTCCCTGATTGGAGACAAGGCTGCCCTCACCATCACAGGGGCACAGACTGAGGATGATGCAATGTATTTCTGTGCTCTATGGTACAGCACCCATTATGTTTTCGGCGGTGGAACCAAGGTCACTGTCCTAGGTCAGCCCAAGTCCACTCCCACTCTCACCGTGTTTCCACCTTCCTCT



ATGGCCTGGATTTCACTTATACTCTCTCTCCTGGCTCTCAGCTCAGGGGCCATTTCCCAGGCTGTTGTGACTCAGGAATCTGCACTCACCACATCACCTGGTGAAACAGTCACACTCACTTGTC